In [1]:
import datetime
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="NDAQ",
    period="5y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [3]:
yfdata = response.copy().dropna()
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-07-03,71.51,72.28,71.49,71.65,65.88,473300
2017-07-05,71.69,72.14,71.54,71.68,65.90,927300
2017-07-06,71.54,71.67,71.16,71.26,65.52,629100
2017-07-07,71.40,71.78,70.94,71.63,65.86,612500
2017-07-10,71.52,71.83,71.32,71.34,65.59,462000
...,...,...,...,...,...,...
2022-06-27,159.81,159.97,157.51,157.93,157.93,749000
2022-06-28,157.93,159.34,154.56,154.66,154.66,644400
2022-06-29,154.55,155.02,151.89,153.06,153.06,774100


In [4]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


# 週足に変換する
def day_2_week(pd: pd.DataFrame) -> pd.DataFrame:
    return (
        pd.copy()
        .resample("W")
        .agg(
            {
                "Open": "first",
                "High": "max",
                "Low": "min",
                "Close": "last",
                "Volume": "sum",
            }
        )
    )


def add_html(file: str, dom: str, first_insert: bool) -> None:
    with open(file) as reader:
        r = reader.read()

    if first_insert:
        r = dom + r
    else:
        r = r + dom

    with open(file, "w") as writer:
        writer.write(r)

In [5]:
def macd_week(data: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    m, s = MACD(day_2_week(yfdata))

    # 週間隔のデータなので、日ごとにする
    df = pd.concat([data.copy(), pd.DataFrame([m, s]).transpose()], axis=1)

    tmp_macd = 0
    tmp_macds = 0

    # 一週間に１日だけしかデータが入っていないので、直近のデータをコピーする
    for index, row in df.iterrows():
        if pd.isna(row["macd"]) or pd.isna(row["macds"]):
            df.at[index, "macd"] = tmp_macd
            df.at[index, "macds"] = tmp_macds
        else:
            tmp_macd = row["macd"]
            tmp_macds = row["macds"]

    df = df.dropna()
    return (df["macd"], df["macds"])


def rsi_week(data: pd.DataFrame, during: int) -> pd.Series:
    rsi = RSI(day_2_week(yfdata), during)
    df = pd.concat([data.copy(), pd.DataFrame([rsi]).transpose()], axis=1)

    tmp_rsi = 0
    key = "rsi_" + str(during)
    for index, row in df.iterrows():
        if pd.isna(row[key]):
            df.at[index, key] = tmp_rsi
        else:
            tmp_rsi = row[key]

    df = df.dropna()
    return df[key]

In [6]:
class My_Strategy(Strategy):
    # RSI
    prop_rsi = 21  # 14
    prop_rsi_high = 66  # 70
    prop_rsi_low = 58  # 30

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.week_macd, self.week_signal = self.I(macd_week, self.data.df)
        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)
        self.rsi_week = self.I(rsi_week, self.data.df, self.prop_rsi)

    def not_trade_with_rsi_range(self):
        return (
            self.prop_rsi_low <= self.rsi_week[-1]
            and self.rsi_week[-1] <= self.prop_rsi_high
        )

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def is_up_trend(self):
        return self.week_macd[-1] > self.week_signal[-1]

    def next(self):
        # 計算できていない場合トレードしない
        if len(self.data.index) < 7 * 26:  # MACD週足の計算に必要な日数
            return

        # 弱いトレンドの際は注文をしない、RSIより判断する
        if self.not_trade_with_rsi_range():
            return

        # 日足MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            self.position.close()
            self.buy()
            return

        # 日足MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            self.position.close()
            self.sell()
            return


bt = Backtest(yfdata, My_Strategy, cash=10000, commission=0.002, exclusive_orders=True)


# 最適化
# dayly : 127%
# optimize = bt.optimize(
#     prop_rsi=[7, 14, 21],
#     prop_rsi_high=range(20, 80, 2),
#     prop_rsi_low=range(20, 80, 2),
#     constraint=lambda p: p.prop_rsi_low < p.prop_rsi_high,
#     method="grid",  # unuse model-based optimization
#     maximize="Equity Final [$]",
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)


# 出力
output = bt.run()
print("Return : " + str(output["Return [%]"]) + "%")
print("Trades : " + str(output["# Trades"]))
print(output)
bt.plot()

Return : 187.41993169677735%
Trades : 44
Start                     2017-07-03 00:00:00
End                       2022-07-01 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                       81.65
Equity Final [$]                     28741.99
Equity Peak [$]                      30052.35
Return [%]                             187.42
Buy & Hold Return [%]                  117.15
Return (Ann.) [%]                       23.53
Volatility (Ann.) [%]                   27.79
Sharpe Ratio                             0.85
Sortino Ratio                            1.57
Calmar Ratio                             1.00
Max. Drawdown [%]                      -23.46
Avg. Drawdown [%]                       -4.32
Max. Drawdown Duration      247 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   44
Win Rate [%]                            52.27
Best Trade [%]                          25.20
Worst Trade [%]                        

Row(id='1713', ...)

In [7]:
# write HTML file
filename = "Return_" + str(round(output["Return [%]"])) + "%"
bt.plot(filename=filename)


add_html(
    filename + ".html",
    pd.DataFrame(
        [
            {
                "rsi": My_Strategy.prop_rsi,
                "rsi_high": My_Strategy.prop_rsi_high,
                "prop_rsi_low": My_Strategy.prop_rsi_low,
                "memo": "週足RSIと日足MACDを組み合わせた売買方法<br>RSIでMACDのダマシを極力除外する<br>具体的にはRSIがhighとlowの間は購入しない",
            }
        ],
        index=["value"],
    )
    .transpose()
    .to_html(escape=False),
    True,
)
add_html(filename + ".html", pd.DataFrame(output).to_html(), False)